<a href="https://colab.research.google.com/github/derek-shing/ML-Engineer-Exam/blob/main/Pytorch/CNNPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
#nn.Conv2d(in_channels = , out_channels, kernel_size=3, stride =2)

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [46]:
train_dataset = torchvision.datasets.CIFAR10(
    root='.',
    train=True,
    transform=transforms.ToTensor(),
    download= True
)

Files already downloaded and verified


In [47]:
test_dataset = torchvision.datasets.CIFAR10(
    root='.',
    train=False,
    transform=transforms.ToTensor(),
    download= True
)

Files already downloaded and verified


In [48]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [49]:
tmp_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 1, shuffle = True)

In [50]:
test_dataset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: .
    Split: Test
    StandardTransform
Transform: ToTensor()

In [51]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: .
    Split: Train
    StandardTransform
Transform: ToTensor()

In [52]:
train_dataset.data.shape

(50000, 32, 32, 3)

In [53]:
#Define Model

class CNN(nn.Module):
  def __init__(self,K):
    super(CNN, self).__init__()

    #define Conv layers
    self.conv1 = nn.Conv2d(3,32, kernel_size=3, stride =2)
    self.conv2 = nn.Conv2d(32,64, kernel_size=3, stride =2)
    self.conv3 = nn.Conv2d(64,128, kernel_size=3, stride =2)

    #define linear layers

    self.fc1 = nn.Linear(128*3*3, 1024)
    self.fc2 = nn.Linear(1024, K)

  def forward(self, x):
      x = F.relu(self.conv1(x))
      x = F.relu(self.conv2(x))
      x = F.relu(self.conv3(x))
      x = x.view(-1 , 128 * 3 * 3)
      x = F.dropout(x, p = 0.5)
      x = F.relu(self.fc1(x))
      x = F.dropout(x, p = 0.2)
      x = self.fc2(x)
      return x



In [54]:
K =10
model = CNN(K)

In [55]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,batch_size = batch_size,shuffle = False)

In [ ]:
for x, y in train_loader:
  print(x.shape)

In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=1152, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [58]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [63]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = []
    for inputs, targets in train_loader:
      inputs, targets = inputs.to(device) , targets.to(device)

      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterion(outputs, targets)

      loss.backward()
      optimizer.step()

      train_loss.append(loss.item())
      
    train_loss = np.mean(train_loss)

    model.eval()
    test_loss =[]
    for inputs, targets in test_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      test_loss = np.mean(loss.item())

    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0
    print(f'Epoch {it+1}/{epochs}, Train Loss : {train_loss:4f}, Test Loss : {test_loss:4f}')
  return train_losses, test_losses


In [64]:
batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs=15)

Epoch 1/15, Train Loss : 1.158493, Test Loss : 1.170672
Epoch 2/15, Train Loss : 1.094077, Test Loss : 1.288013
Epoch 3/15, Train Loss : 1.040560, Test Loss : 0.889964
Epoch 4/15, Train Loss : 0.989981, Test Loss : 0.974922
Epoch 5/15, Train Loss : 0.947933, Test Loss : 0.874247
Epoch 6/15, Train Loss : 0.902622, Test Loss : 0.774446
Epoch 7/15, Train Loss : 0.866063, Test Loss : 0.830383
Epoch 8/15, Train Loss : 0.837053, Test Loss : 1.271447
Epoch 9/15, Train Loss : 0.809275, Test Loss : 1.547425
Epoch 10/15, Train Loss : 0.779408, Test Loss : 1.084141
Epoch 11/15, Train Loss : 0.745080, Test Loss : 0.816144
Epoch 12/15, Train Loss : 0.720127, Test Loss : 0.949359
Epoch 13/15, Train Loss : 0.706278, Test Loss : 1.224684
Epoch 14/15, Train Loss : 0.686247, Test Loss : 1.365459
Epoch 15/15, Train Loss : 0.655162, Test Loss : 1.250355


(array([1.15849268, 1.09407671, 1.0405602 , 0.98998118, 0.94793335,
        0.90262246, 0.86606345, 0.83705298, 0.809275  , 0.7794079 ,
        0.74508017, 0.72012701, 0.70627801, 0.68624696, 0.65516194]),
 array([1.1706723 , 1.28801298, 0.88996357, 0.9749223 , 0.87424737,
        0.77444571, 0.8303833 , 1.27144682, 1.54742491, 1.08414066,
        0.81614363, 0.9493593 , 1.22468436, 1.36545885, 1.25035489]))